# Batch Propogation of TLEs we obtained from Space Track
In this notebook we will propogate the TLEs we obtained from space track using the sgp4 propogator. However we will make use of the batch propogator that incorporates pytorch methods and tensors to speed up the porpogation process of TLEs

## Dependencies

In [45]:
import json
import pandas as pd
import dsgp4
import numpy as np
import torch

## Read the TLEs from the JSON file and make them into objects

In [ ]:
def raj_TLE_json_to_array_TLE_ojects(filename):

    """
    This method takes in a json file with satellite data from Space track, and returns an array of TLE objects using
    dsgp4.tle.TLE()

    Parameters:
    ----------------
    filename (str): the json file name which was obtained from Space track

    Returns:
    ----------------
    TLE_arr (numpy array): A numpy array with each of the TLEs converted into TLE objects using dsgp4.tle.TLE()

    """
    
    # Read the file
    with open(filename, 'r') as file:
        tle_data = json.load(file)

    # Create a pandas dataframe with the data
    df_tle = pd.DataFrame(tle_data)

    # Create the full TLE string by combining line 1 and line 2, separated by a newline
    df_tle["TLE"] = df_tle['TLE_LINE1'] + '\n' + df_tle['TLE_LINE2']

    TLE_lst = []

    for TLE in df_tle['TLE']:
        TLE_lst.append(dsgp4.tle.TLE(TLE))

    TLE_arr = np.array(TLE_lst)

    return TLE_arr


TLE_arr = raj_TLE_json_to_array_TLE_ojects('Starlink_Satellites.json')

## Batch Propogation

In [1]:
def raj_batch_propogate(TLE_arr,propogation_time,steps,TLEs_limit = len(TLE_arr)):
    """
    This method takes in an array of TLEs and returns a
    dsgp4.tle.TLE()

    Parameters:
    ----------------
    filename (str): the json file name which was obtained from Space track

    Returns:
    ----------------
    TLE_arr (numpy array): A numpy array with each of the TLEs converted into TLE objects using dsgp4.tle.TLE()

    """


    tles_ = []
    for tle in TLE_arr[0:TLEs_limit]:
        tle_ += [tle]*steps
    
    tsinces = torch.cat([torch.linspace(0,propogation_time)]*len(TLE_arr))
    # initiate the TLESs
    _,tle_batch = dsgp4.initialize_tle(tles_)

    states_teme = dsgp4.propagate_batch(tle_batch,tsinces)

    return states_teme



tles_=[]
for tle in TLE_arr[0:100]: #copy each tle multiple times for each epoch to calculate 
    tles_+=[tle]*10000


tsinces = torch.cat([torch.linspace(0,24*60,10000)]*len(TLE_arr[0:100]))
#first let's initialize them:
_,tle_batch=dsgp4.initialize_tle(tles_)


#we propagate the batch of 3,000 TLEs for 1 day:
states_teme=dsgp4.propagate_batch(tle_batch,tsinces)

type(states_teme)

NameError: name 'TLE_arr' is not defined

In [64]:
states_teme[0]

tensor([[-2.8978e+03,  2.7173e+03,  5.1754e+03],
        [-5.9448e+00, -5.0241e+00, -6.8608e-01]])